In [ ]:
import numpy as np

############ OLD! use version from registration.py #############
# first try, 2d affine test

# TODO: use version from registration.py -> make package

import numpy as np

# first try, 2d affine test

from registration import *



make_coeff_affine2d_fixed(0,0)

In [ ]:
# generate some rotated points

a = np.pi / 16
a2 = np.pi / 12
rot = np.array([[np.cos(a), -np.sin(a)],[np.sin(a), np.cos(a)]])
rot2 = np.array([[np.cos(a2), -np.sin(a2)],[np.sin(a2), np.cos(a2)]])
p1 = [np.random.uniform(size=2) for _ in range(32)]
p2 = [rot.dot(p) for p in p1]
p3 = [rot2.dot(p) for p in p2]



from matplotlib import pyplot as plt
%matplotlib inline

plt.plot([p[0] for p in p1], [p[1] for p in p1], '.')
plt.plot([p[0] for p in p2], [p[1] for p in p2], '.')
plt.plot([p[0] for p in p3], [p[1] for p in p3], '.')

In [ ]:
coeffs = []
ys = []

for (xf, x) in zip(p1, p2):
    c, y = make_coeffs_affine_nd_fixed(xf, x, 2, 0, 2)
    coeffs.append(c)
    ys.append(y)

for (x1, x2) in zip(p2, p3):
    c, y = make_coeffs_affine_nd(x1, x2, 2, 0, 1, 2)
    coeffs.append(c)
    ys.append(y)


coeffs = np.vstack(coeffs)
ys = np.concatenate(ys)

r, _, _, _ = np.linalg.lstsq(coeffs, ys, rcond=None)

rot

print(r[6:].reshape((2,3)))
print(np.linalg.inv(rot2.dot(rot)))

In [ ]:
tr1 = np.array([12,124.2])
tr2 = np.array([-1,14.2])
p1 = [np.random.uniform(size=2) for _ in range(50000)]
p2 = [p + tr1 for p in p1]
p3 = [p + tr2 + np.random.uniform(-2,2,2) for p in p2]

coeffs = []
ys = []

for (xf, x) in zip(p1, p2):
    c, y = make_coeffs_translation_nd(xf, x, 2, 0, 1, 3)
    coeffs.append(c)
    ys.append(y)

for (x1, x2) in zip(p2, p3):
    c, y = make_coeffs_translation_nd(x1, x2, 2, 1, 2, 3)
    coeffs.append(c)
    ys.append(y)


coeffs = np.vstack(coeffs)
ys = np.concatenate(ys)

r, _, _, _ = np.linalg.lstsq(coeffs, ys, rcond=None)

rot

print(r[4:] - r[:2])
print(-(tr2+tr1))

In [ ]:
import re
from collections import defaultdict
from io import BytesIO
from sys import stdout
from xml.etree.ElementTree import ElementTree, Element, SubElement, dump
from xml.dom.minidom import parse

import numpy as np
import pandas as pd
from scipy.spatial import KDTree


def add_array_set(l, a, id_a):
    """
    add array to list of arrays, only keep unique
    return index of added or kept array
    """
    for id_b, b in enumerate(l):
        if np.allclose(a, b):
            return id_b
    l += [a]
    return id_a

# collect list of pixel coordinates per pair
res = {}

# accumulate manual click output from multiple files
content = ''
files = (
    '/Volumes/davidh-ssd/manual_ips_angle1_v1.txt',
    '/Volumes/davidh-ssd/manual_ips_angle2.txt',
    '/Volumes/davidh-ssd/manual_ips_mview.txt',
)
for file in files:
    with open(file, 'r') as fd:
        content += '\n' + fd.read()

# split on pair header : !vid_a-vid_b
pair_cts = content.split('\n!')
p_header = re.compile('!*([0-9]+)-([0-9]+)')

for pair_ct in pair_cts:

    # ignore empty or commented-out content
    if pair_ct.strip() == '' or pair_ct.strip().startswith('#'):
        continue
    
    # get vid-pair
    header = pair_ct.strip().split('\n', 1)[0]
    vid_a, vid_b = p_header.match(header.strip()).groups()
    
    res_pair = []
    pa = re.compile('.*?tpId=0 setupId={}--- global: (\(.*?\))--- pixel: (\(.*?\)).*?'.format(vid_a))
    pb = re.compile('.*?\n?tpId=0 setupId={}--- global: (\(.*?\))--- pixel: (\(.*?\)).*?'.format(vid_b))
    
    # split on --- lines
    lines = pair_ct.split('---\n')[1:]
    
    # go over pairs of file chunks
    for line_a, line_b in zip(*[iter(lines)]*2):
        
        try:
            # parse global, pixel coords
            gla, pxa = pa.match(line_a.strip()).groups()
            glb, pxb = pb.match(line_b.strip()).groups()
        except AttributeError as ex:
            print('Error parsing {} on:\n{}\n{}'.format((vid_a, vid_b), line_a, line_b))
            continue
            
        # add as pair of np-arrays
        res_pair += [(np.array([*map(float, pxa.strip('()').split(','))]),
                      np.array([*map(float, pxb.strip('()').split(','))]))]
    
    # add all for pair
    res[(int(vid_a), int(vid_b))] = res_pair

In [ ]:
# parameters for outlier removal
# roughly based on http://docs.pointclouds.org/1.7.1/classpcl_1_1_statistical_outlier_removal.html#details
sd_mult_t = 3.0 # how many sds pixel-click pairs may devate from mean until they are removed
p_estim = 0.8 # quantile of pixel-click pairs to use for mean, sd estimation

# format string for MVR interest point files 
ip_fstring = '/Volumes/davidh-ssd/BS_TEST/interestpoints/tpId_0_viewSetupId_{}.beads.ip.txt'

res_ip = {}
for vid_a, vid_b in res.keys():

    # build kdtrees for real ips
    df_a = pd.read_csv(ip_fstring.format(vid_a), sep='\t')
    locs_a = np.array(df_a[['x', 'y', 'z']])
    kd_a = KDTree(locs_a)
    df_b = pd.read_csv(ip_fstring.format(vid_b), sep='\t')
    locs_b = np.array(df_b[['x', 'y', 'z']])
    kd_b = KDTree(locs_b)

    res_pair = [] # pixel point pairs from IPs
    manual_pair = [] # pixel point pairs from clicks
    # find closest neighbours
    for point_a, point_b in res[(vid_a, vid_b)]:
        d_a, idx_a = kd_a.query(point_a)
        d_b, idx_b = kd_b.query(point_b)
        # keep pair only if we find reasonable match (distance of both < d_thresh)
        # NB: removed for statistical outlier removal
        #if (d_a < d_thresh and d_b < d_thresh):
        res_pair += [(locs_a[idx_a], locs_b[idx_b])]
        manual_pair += [(point_a, point_b)]

    # filter outliers    
    # vid_a: get mean and sd of pixel-clicked coordinates
    ds = [l1 - l2 for (l1,_),(l2,_) in zip(res_pair, manual_pair)]
    mu, sd = (np.mean(np.array(sorted(ds, key= lambda p: np.linalg.norm(p))[:int(len(ds)*p_estim)]), axis=0),
              np.std(np.array(sorted(ds, key= lambda p: np.linalg.norm(p))[:int(len(ds)*p_estim)]), axis=0))
    
    # remember indices of good points
    idxes_good = set([idx for idx, d in enumerate(ds) if np.all(np.abs(mu-d) < sd * sd_mult_t)])

    # same for vid_b
    ds = [l1 - l2 for (_, l1),(_, l2) in zip(res_pair, manual_pair)]
    mu, sd = (np.mean(np.array(sorted(ds, key= lambda p: np.linalg.norm(p))[:int(len(ds)*p_estim)]), axis=0),
              np.std(np.array(sorted(ds, key= lambda p: np.linalg.norm(p))[:int(len(ds)*p_estim)]), axis=0))
    
    # good points for both views
    idxes_good &= set([idx for idx, d in enumerate(ds) if np.all(np.abs(mu-d) < sd * sd_mult_t)])
    
    # keep filtered IP coordinate pairs
    res_pair_filt = [rp for idx,rp in enumerate(res_pair) if idx in idxes_good] 
    res_ip[(vid_a, vid_b)] = res_pair_filt

# quick check: how many pairs remain?
for k, v in res_ip.items():
    print(k, len(v))

In [ ]:
angle1_idxs = list(range(6)) + list(range(12,18))
res2_rot = {(i1, i2):  list(zip(
          [p[0] if i1 in angle1_idxs else p[0] * np.array([-1,1,-1]) for p in v],
          [p[1] if i2 in angle1_idxs else p[1] * np.array([-1,1,-1]) for p in v])) for (i1,i2) ,v in res_ip.items()}

res_ip2 = {k: ([p[0] for p in v], [p[1] for p in v]) for k,v in res2_rot.items()}

from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook
reg = register_affine(res_ip2, [0])

from collections import defaultdict
transformed = defaultdict(list)
err = defaultdict(list)
for (i1, i2), pairs in res2_rot.items():
    for p1, p2 in pairs:
        t1 = aug_mat(reg[i1]).dot(aug_vec(p1))[:3]
        t2 = aug_mat(reg[i2]).dot(aug_vec(p2))[:3]
        transformed[i1].append(t1)
        transformed[i2].append(t2)
        err[(i1, i2)].append(np.linalg.norm(t1-t2))
        

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='3d')
for ps in transformed.values():
    xs, ys, zs = [], [], []
    for d in ps:
        xs.append(d[0])
        ys.append(d[1])
        zs.append(d[2])
        ax.scatter(xs, ys, zs, alpha=0.5)
plt.show()

for k,v in err.items():
    print(k, np.mean(v))
    
np.mean([np.mean(v) for k,v in err.items()])